This one cleans all files in pre1790 debt certs (ie. who the gov needs to pay back and when)
90th is a cent (ie. instead of 100 cents per dollar, 90 cents per dollar)
Aggregated columns: "state", "date_issued", "title", "fname", "lname", "deceased", "date_due", "dollar", "90th", "specie", "striked", "note"

In [5]:
#Imports
import pandas as pd
import datetime
import numpy as np
import json
import os

In [6]:
#Helpers
def deNaN(series):
    """
    amends pandas series by replacing NaN values with empty strings
    :param series: pandas series
    """

    return series.apply(lambda x: "" if type(x) != str else x)

In [7]:
#Load and prune the top metadata of the docs

#NJ: mon | day | yr | title | fname | lname | mon | day | yr | $ | 90th | strike | note
nj = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_NJ.xlsx")
nj = nj[10:5117].drop(columns=["Record Name", "Records of the New Jersey Continental Loan Office, 1777-1790", "Unnamed: 2", "Unnamed: 9", "F", "Unnamed: 11", "Unnamed: 19"])

#NY: mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th | strike | note | notes
ny = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_NY.xlsx")
ny = ny[11:7319].drop(columns=["Record Name", "Records of the New Jersey and New York Continental Loan Offices 1777-1790", "Unnamed: 2", "Unnamed: 3", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12"])

#NH: mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th | strike | note | notes
nh = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_NH.xlsx")
nh = nh[11:190].drop(columns=["Record Name", "Records of the Connecticut, New Hampshire, and Rhode Island Continental Loan Office, 1777-1789", "Unnamed: 2", "Unnamed: 3"])

#MA: mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th | strike | note
ma = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_MA.xlsx")
ma = ma[11:2090].drop(columns=["Record Name", "Records of the Massachusetts Continental Loan Office, 1777-1791", "Unnamed: 2", "Unnamed: 3", "Unnamed: 20"])

#DE: day | mon | yr | title | fname | lname | day | mon | yr | $ | 90th | strike | note
de = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_DE.xlsx")
de = de[11:637].drop(columns=["Record Name", 
    "Mr Paterson remarks that the certificate No. 57 says interest payable from the 1st of March 1780 instead of 1st May 1780, 123 From the 1st of January 1780 instead of 14th January, 234 is in the name of James James    John James", "Unnamed: 2"])

#CT: mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th | strike | note | notes
ct = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_CT.xlsx")
ct = ct[11:1095].drop(columns=["Record Name", "Records of the Connecticut, New Hampshire, and Rhode Island Continental Loan Office, 1777-1789", "Unnamed: 2", "Unnamed: 3", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12", "Unnamed: 19"])

#PA_stelle: yr | mon | day | title | fname | lname | yr | mon | day | $ | 90th | strike | notes
pa1 = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_PA_stelle.xlsx")
pa1 = pa1[11:9221].drop(columns=["Record Name", "Benjamin Steller commisisioner for Pennsylvania Register NO. 1. From No. 1 to 4403", "Unnamed: 2", "Unnamed: 3", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12"])

#PA_story: yr | mon | day | title | fname | lname | yr | mon | day | $ | 90th | specie amount (2 cols) | strike | notes
pa2 = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_PA_story.xlsx")
pa2 = pa2[11:966].drop(columns=["Record Name", "Records of the Pennsylvania Continental Loan Office, 1785-1786", "Unnamed: 2", "Unnamed: 3", "Unnamed: 15", "Unnamed: 16"])

#RI: mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th
ri = pd.read_excel("../../data_raw/pre1790/liquidated_debt_certificates_RI.xlsx")
ri = ri[11:1537].drop(columns=["Record Name", "Records of the Connecticut, New Hampshire, and Rhode Island Continental Loan Office, 1777-1789", "Unnamed: 2", "Unnamed: 3", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12", "Unnamed: 18"])

#NOT STATE ONES
#marine - the word "Reg" shows up in majority of rows in the left total dollars column - will need to get rid of those
#mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th | total dollars (2 cols) | strike? | note | notes
marine = pd.read_excel("../../data_raw/pre1790/Marine_Liquidated_Debt_Certificates.xlsx")
marine = marine[11:776].drop(columns=["Record Name", "Registers of Liquidated Debt Certificates for the Marine Department, 1786-89", "Unnamed: 2", "Unnamed: 3"])

#pierce - Last, first, value, state
pierce = pd.read_excel("../../data_raw/pre1790/Pierce_Certs_cleaned_2019.xlsx")
pierce = pierce[1:93308].drop(columns=["CN", "Group", "To Whom Issued", "Officer"])

#office - State (# 1-9), yr, mon, day, title, fn, ln, dollar, specie
office = pd.read_excel("../../data_raw/pre1790/loan_office_certificates_9_states.xlsx")
office = office[1:80913].drop(columns=["Title 2", "First Name 2", "Last Name 2"])

#aggregate data table - im making the dates into a format readable and writable by datetime module
#"state", "date_issued", "title", "fname", "lname", "deceased", "date_due", "dollar", "90th", "specie", "striked", "note" -- specie is optional, very few files have it but we should probably keep it anyway
agg_df = pd.DataFrame()

/Users/liamloughead/Code/SPEOC-pt-1/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/liamloughead/Code/SPEOC-pt-1/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/liamloughead/Code/SPEOC-pt-1/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/liamloughead/Code/SPEOC-pt-1/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/liamloughead/Code/SPEOC-pt-1/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/liamloughead/Code/SPEOC-pt-1/.venv/lib/python3.11/site-packages

Cleaning process:
1. fix all records (ie. remove extraneous information + combine them into agg)
   - possibilites of stuff to remove: "Estate of", "Heirs of",
   - possibilites of stuff to fix (ie. find the right name for): "and Co.", "Owners of",
   - first or last name missing: either make the other one undefined, or the entire name is in the first or last name column and it needs to be spread out
   - occupation could be in the name
   - some last names have spaces in them (specifically Van) - replace them with spaces
   - so it needs to look like:
     - if fname split or lname split != 1:
       - if its just lname contains van, replace "Van " with "Van-"
       - check and auto remove Estate of, Heirs of
       - prompt for "Owners of" + "and Co."

In [138]:
#corrections.json file format:
# [
#     {
#         "state_code": "nj",
#         "i": 1,
#         "name_correct": "",
#         "date_correct": 38945783 #ordinal
#     }
# ]

enable_manual_corrections = input("Enable manual correction system (yes, no)? (DO NOT ENABLE IF YOU ARE NOT READY TO MAKE MANUAL CORRECTIONS) > ")
enable_manual_corrections = True if enable_manual_corrections == "yes" else False

def retrieve_manual_correction(state_code, index):
    if not os.path.exists("corrections.json"):
        f = open("corrections.json", "w")
        f.write("[]")
        f.close()
    file = json.load(open("corrections.json", "r"))
    for obj in file:
        if obj["state_code"] == state_code and obj["i"] == index:
            return (obj["name_correct"], obj["date_correct"])
    return None

def save_manual_correction(state_code, index, name_correct, date_correct):
    if not enable_manual_corrections: return
    if not os.path.exists("corrections.json"):
        f = open("corrections.json", "w")
        f.write("[]")
        f.close()
    file = json.load(open("corrections.json", "r"))
    file.append({
        "state_code": state_code,
        "i": index,
        "name_correct": name_correct,
        "date_correct": date_correct
    })
    f = open("corrections.json", "w")
    json.dump(file, f)
    f.close()

def process_date(yr, mon, day, i: bool, state_code, index):
    try:
        d = datetime.date(int(yr), int(mon), int(day))
        return (d.toordinal(), False)
    except Exception as e:
        if "10: ''" in str(e): #ie. the "Invalid literal for base 10: ''" error, which means blank, which means just make it 0
            return (0, False)
        manual = retrieve_manual_correction(state_code, index)
        if manual == None:
            if 'month must' in str(e): #ie. month must be in range 1..12 - just swap month and day
                d = datetime.date(yr, day, mon)
                return (d.toordinal(), False)
            new = input(f"{state_code}: {'RE, ' if ('range' in str(e)) else ''}{'Issued: ' if i else 'Expiries: '} {yr} {mon} {day} (yr-mon-day):")
            if new == "" and i == False:
                return (0, False)
            d = datetime.date(int(new.split()[0]), int(new.split()[1]), int(new.split()[2]))
            return (d.toordinal(), True)
        else:
            return (int(manual[1].split('-')[0]) if i else int(manual[1].split('-')[1]), False)

In [143]:
def check_note_present(note):
    if note != "":
        if "Lo" in note or "Ro" in note:
            return False
        try:
            float(note.strip())
            return False
        except:
            pass
        return True
    return False

def process(i, state_code, mon, day, yr, title, fname, lname, dmon, dday, dyr, dollar, specie, ninety, strike, note):
    title, fname, lname = title.strip(), fname.strip(), lname.strip()
    #Replace anything that could come up
    lname = lname.replace("Van ", "Van-")
    lname = lname.replace(" Van", "-Van")
    fname = fname.replace("Van ", "Van-")
    fname = fname.replace(" Van", "-Van")
    fname = fname.replace("Estate of ", "").replace("Heirs of ", "").replace("Heir of ", "")
    fname = fname.replace(" & Co", "").replace(" & co", "").replace(" and Co", "").replace(" and co", "")
    lname = lname.replace(" & Co", "").replace(" & co", "").replace(" and Co", "").replace(" and co", "")
    if len(fname.split()) == 1 and lname == "": lname = "undefined" # if there is no last name, make it undefined
    if len(fname.split()) == 2:
        if len(fname.split()[1]) == 1: fname = fname.split()[0] # if middle initial in fname, drop it
        elif len(fname.split()[1]) > 3: fname = fname.replace(" ", "-") # usually means 2 names
    if len(lname.split()) == 2:
        if len(lname.split()[1]) == 1: lname = lname.split()[0] # if initial in lname, drop it
        elif len(lname.split()[1]) > 3: lname = lname.replace(" ", "-") # usually means 2 names
    if fname == "" and lname == "": return None # Drop ones with no name data
    note = "" if "not sure" in str(note) else str(note)
    note = "" if "RO" in str(note) else str(note)
    note = "" if "ro" in str(note) else str(note)
    try: # drop notes that are just numbers
        for ns in note.split("."):
            int(ns)
        note = ""
    except:
        pass
    
    isdeceased = "deceased" in fname or "deceased" in lname
    if ('&' in fname) or (' and' in fname) or ('|' in fname):
        #Means there is co-ownership
        to_add = []
        sepr = ""
        if ("&" in fname): sepr = "&"
        if (" and" in fname): sepr = "and"
        if ("|" in fname): sepr = "|"
        idate, iman = process_date(yr, mon, day, True, state_code, i)
        expdate, eman = process_date(dyr, dmon, dday, False, state_code, i)
        if iman or eman: save_manual_correction(state_code, i, "", str(idate) + "-" + str(edate))
        for i in range(len(fname.split(sepr))):
            to_add.append(pd.Series([
                state_code,
                idate,
                title, fname.split(sepr)[i].strip(), "undefined",
                isdeceased,
                expdate,
                dollar, ninety, specie, strike
            ]))
        return to_add
    idate, iman = process_date(yr, mon, day, True, state_code, i)
    edate, eman = process_date(dyr, dmon, dday, False, state_code, i)
    if iman or eman: save_manual_correction(state_code, i, "", str(idate) + "-" + str(edate))
    if len(fname.split()) != 1 or len(lname.split()) != 1 or check_note_present(note):
        correction = retrieve_manual_correction(state_code, i)
        if correction == None:
            corrected = input(f"{state_code}, Title: '{title}' F: '{fname}' L: '{lname}' ({note})> ")
            if corrected == "n":
                save_manual_correction(state_code, i, "DROP", str(idate) + "-" + str(edate))
                return None
            if corrected == "s": return 's'
            if corrected == "c": corrected = title + " " + fname + " " + lname
            if len(corrected.split()) == 2:
                fname, lname = corrected.split()
            if len(corrected.split()) == 3:
                title, fname, lname = corrected.split()
            save_manual_correction(state_code, i, corrected, str(idate) + "-" + str(edate))
        else:
            if correction[0] == "DROP": return None
            if len(correction[0].split()) == 2:
                fname, lname = correction[0].split()
            if len(correction[0].split()) == 3:
                title, fname, lname = correction[0].split()
            print(correction[1].split("-"))
            idate, edate = correction[1].split("-")
            idate, edate = int(idate), int(edate)
    return [pd.Series([
        state_code,
        idate,
        title, fname, lname,
        isdeceased,
        edate,
        dollar, ninety, specie, strike
    ])]

In [ ]:
#Aggregate - loop through each dataframe, run the process function on it, then add it to agg_df
dfs = {"nj": nj, "nh": nh, "ny": ny, "ma": ma, "de": de, "ct": ct, "pa1": pa1, "pa2": pa2, "ri": ri}

for state in dfs.keys():
    df = dfs[state]
    for index, row in df.iterrows():
        row = row.replace(np.nan,'',regex=True)
        out = None
        if state == 'nj':
            # mon | day | yr | title | fname | lname | mon | day | yr | $ | 90th | strike | note
            out = process(index, state,
                row[0], row[1], row[2], row[3], row[4],
                row[5], row[6], row[7], row[8], row[9], "",
                row[10], row[11], row[12])
        if state == 'ny' or state == 'nh' or state == "ct":
            # mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th | strike | note | notes
            out = process(index, state,
                row[0], row[1], row[2], row[5], row[4],
                row[3], row[6], row[7], row[8], row[9], "",
                row[10], row[11], row[12])
        if state == 'ma':
            #mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th | strike | note
            out = process(index, state,
                row[0], row[1], row[2], row[5], row[4],
                row[3], row[6], row[7], row[8], row[9], "",
                row[10], row[11], row[12])
        if state == "de":
            #day | mon | yr | title | fname | lname | day | mon | yr | $ | 90th | strike | note
            out = process(index, state,
                row[1], row[0], row[2], row[3], row[4],
                row[5], row[7], row[6], row[8], row[9], "",
                row[10], row[11], row[12])
        if state == "pa1":
            # yr | mon | day | title | fname | lname | yr | mon | day | $ | 90th | strike | notes
            out = process(index, state,
                row[1], row[2], row[0], row[3], row[4],
                row[5], row[7], row[8], row[6], row[9], "",
                row[10], row[11], row[12])
        if state == "pa2":
            # yr | mon | day | title | fname | lname | yr | mon | day | $ | 90th | specie amount (2 cols) | strike | notes
            out = process(index, state,
                row[1], row[2], row[0], row[3], row[4],
                row[5], row[7], row[8], row[6], row[9], row[11] + "." + row[12],
                row[10], row[13], row[14])
        if state == "ri":
            # mon | day | yr | fname | lname | title | mon | day | yr | $ | 90th
            out = process(index, state,
                row[0], row[1], row[2], row[5], row[4],
                row[3], row[6], row[7], row[8], row[9], "",
                row[10], "", "")
        if out == None: continue
        if out == "s":
            print("manual stop")
            break
        for new_row in out:
            agg_df = pd.concat([agg_df, new_row.to_frame().T], ignore_index=True)
agg_df

In [145]:
#Clear the agg_df
agg_df = agg_df.iloc[0:0]